# Домашнее задание № 3. Исправление опечаток

## 1. Учет грамматики при оценке исправлений (3 балла)

В последнюю итерацию алгоритма для генерации исправлений добавьте еще один компонент - учет грамматической информации. Частично она уже учитывается за счет языковой модели (вероятность предсказывается для словоформы), но такой подход ограничен из-за того, что модель не может ничего предсказать для словоформ, которых не было в обучающей выборке. Чтобы это исправить постройте еще одну "языковую модель" на грамматических тэгах:
1) Используя mystem или pymorphy, разметьте какой-нибудь корпус (например, кусок wiki из семинара) или воспользуйтесь уже размеченным корпусом (например, opencorpora)
2) соберите униграмные и биграмные статистики на уровне грамматических тэгов (например, вместо `задача важна` у вас будет биграм `S,жен,неод=им,ед A=ед,кр,жен`). Для простоты можете начать только с частеречных тэгов и добавить остальную информацию позже
3) напишите функцию, которая будет оценивать вероятность данного предложения на основе грамматической языковой модели (статистик из предыдущего шага). Функция должна сначала преобразовать текст в грамматические тэги, используя точно такой же подход, что использовался на шаге 1. 
4) в функции correct_text_with_lm замените compute_sentence_proba на вашу новую функцию и прогоните получившийся алгоритм на данных
5) сравните предсказания с предсказанием изначального correct_text_with_lm, проверьте метрики и посмотрите на различие в ошибках и исправлениях, найдите несколько примеров отличий в предсказаниях этих подходов

In [2]:
import os
import re
import numpy as np
import stanza
import textdistance
from collections import Counter, defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
import itertools
from tqdm.notebook import tqdm


stanza.download('ru')

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: ru (Russian) ...


INFO:stanza:Downloaded file to /root/stanza_resources/ru/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources


### 1. Загрузка данных

Я решил использовать уже размеченный корпус в качестве словаря (и для слов, и для тегов) — а именно, Sintagrus. Он размечен в UD, поэтому для разметки наших данных с опечатками мы будем использовать Stanza — там русская модель натренирована на том же самом корпусе, то есть результат будет максимально консистентный.

In [3]:
# Синтагрус
!wget -q https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
!wget -q https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-test.conllu
!wget -q https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
!wget -q https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-b.conllu

# Данные с опечатками
!wget -q https://github.com/mannefedov/compling_nlp_hse_course/raw/master/notebooks/spelling/data.zip
!unzip -o -q data.zip

### 2. Парсим корпус и собираем статистики

В качестве тегов для биграмм будем использовать и часть речи, и все фичи (только значения, без названий), отсортированные по алфавиту. Соединим это всё запятой.

In [4]:
# Список файлов корпуса
conllu_files = [
    'ru_syntagrus-ud-dev.conllu',
    'ru_syntagrus-ud-test.conllu',
    'ru_syntagrus-ud-train-a.conllu',
    'ru_syntagrus-ud-train-b.conllu'
]

vocab = Counter()
word_unigrams = Counter()
word_bigrams = Counter()
tag_unigrams = Counter()
tag_bigrams = Counter()

In [5]:
def parse_conllu_features(pos, feats_str):
    # Если фич нет, возвращаем только POS
    if feats_str == '_' or not feats_str:
        return pos

    feats = [f.split('=') for f in feats_str.split('|') if '=' in f]
    feats.sort(key=lambda x: x[0])  # Сортировка по имени фичи (Key)

    feat_values = [f[1] for f in feats]

    return pos + ',' + ','.join(feat_values)


def get_ngrams(sequence, n=2):
    return [' '.join(sequence[i:i + n]) for i in range(len(sequence) - n + 1)]

In [6]:
for filepath in conllu_files:
    with open(filepath, 'r', encoding='utf-8') as f:
        current_sent_words = []
        current_sent_tags = []

        for line in f:
            line = line.strip()

            # Пропускаем комментарии
            if line.startswith('#'):
                continue

            # Пустая строка - конец предложения
            if not line:
                if current_sent_words:
                    # Добавляем маркеры начала и конца
                    full_words = ['<start>'] + current_sent_words + ['<end>']
                    full_tags = ['<start>'] + current_sent_tags + ['<end>']

                    # Обновляем статистики
                    # Словарь без спецсимволов
                    vocab.update(current_sent_words)
                    word_unigrams.update(full_words)
                    word_bigrams.update(get_ngrams(full_words))

                    tag_unigrams.update(full_tags)
                    tag_bigrams.update(get_ngrams(full_tags))

                    current_sent_words = []
                    current_sent_tags = []
                continue

            # Парсинг токена
            parts = line.split('\t')
            if len(parts) != 10:
                continue

            word = parts[1].lower()
            pos = parts[3]
            feats = parts[5]

            # Игнорируем пунктуацию
            if pos == 'PUNCT':
                continue

            # Формируем тэг
            full_tag = parse_conllu_features(pos, feats)

            current_sent_words.append(word)
            current_sent_tags.append(full_tag)

In [7]:
# Подсчет общего количества для вероятностей
N_words = sum(word_unigrams.values())
N_tags = sum(tag_unigrams.values())

print(f"Размер словаря слов: {len(vocab)}")
print(f"Пример биграммы тегов: {list(tag_bigrams.most_common(10))}")

Размер словаря слов: 121389
Пример биграммы тегов: [('<start> ADP', 10846), ('<start> CCONJ', 6913), ('ADJ,Gen,Pos,Fem,Sing NOUN,Inan,Gen,Fem,Sing', 6088), ('ADP NOUN,Inan,Loc,Masc,Sing', 5255), ('<start> ADV,Pos', 5022), ('ADJ,Gen,Pos,Masc,Sing NOUN,Inan,Gen,Masc,Sing', 4877), ('ADP NOUN,Inan,Loc,Fem,Sing', 4689), ('ADJ,Nom,Pos,Fem,Sing NOUN,Inan,Nom,Fem,Sing', 4417), ('NOUN,Inan,Gen,Fem,Sing <end>', 4101), ('ADJ,Nom,Pos,Masc,Sing NOUN,Inan,Nom,Masc,Sing', 3949)]


### 3. Векторизация

In [8]:
word2id = list(vocab.keys())
id2word = {i: word for i, word in enumerate(word2id)}

In [9]:
# Векторизация по символам (1-3 граммы)
vec = CountVectorizer(analyzer='char', max_features=10000, ngram_range=(1, 3))
X = vec.fit_transform(word2id)

print(f"Размерность матрицы: {X.shape}")

Размерность матрицы: (121389, 10000)


### 4. Препроцессинг тестовых данных

Stanza самостоятельно умеет токенизировать, поэтому эти токены мы будем использовать для выравнивания, игнорируя только пунктуацию. При этом мы передаём все предложения разом, в виде одного документа, и отключаем в токенизаторе деления на предложения (то есть указываем, что текст на предложения уже разбит) — POS-теггинг и присвоение морфологических фич тогда выполнятся относительно быстро, правда, только на GPU.

In [10]:
nlp_full = stanza.Pipeline(
    lang='ru',
    tokenize_no_ssplit=True,
    verbose=False)
nlp_tokenize = stanza.Pipeline(
    lang='ru',
    processors='tokenize',
    tokenize_no_ssplit=True,
    verbose=False)

In [11]:
with open('sents_with_mistakes.txt', 'r', encoding='utf-8') as f:
    bad_lines = f.read().splitlines()

with open('correct_sents.txt', 'r', encoding='utf-8') as f:
    true_lines = f.read().splitlines()

bad_text_formatted = "\n\n".join(bad_lines)
true_text_formatted = "\n\n".join(true_lines)

In [12]:
# Обработка текстов с опечатками
doc_bad = nlp_full(bad_text_formatted)
doc_true = nlp_tokenize(true_text_formatted)

print(f"Processed bad sentences: {len(doc_bad.sentences)}")
print(f"Processed true sentences: {len(doc_true.sentences)}")

Processed bad sentences: 915
Processed true sentences: 915


### 5. Всякие полезные функции с семинара

In [13]:
def get_closest_match_with_metric(
        text,
        lookup,
        topn=20,
        metric=textdistance.levenshtein):
    similarities = Counter()
    for word in lookup:
        similarities[word] = metric.similarity(text, word)
    return similarities.most_common(topn)


def get_closest_match_vec(text, X, vec, topn=20):
    # Векторизуем входное слово
    v = vec.transform([text])
    # Считаем косинусное расстояние до всех слов словаря
    similarities = cosine_distances(v, X)[0]
    # Берем индексы топ-N ближайших
    top_indices = similarities.argsort()[:topn]
    return [(id2word[idx], similarities[idx]) for idx in top_indices]


def get_closest_hybrid_match(
        text,
        X,
        vec,
        topn=3,
        metric=textdistance.damerau_levenshtein):
    # Отбираем кандидатов по косинусному расстоянию (с запасом *4)
    candidates = get_closest_match_vec(text, X, vec, topn * 4)
    lookup = [cand[0] for cand in candidates]

    # Ранжируем их по метрике (Дамерау-Левенштейн)
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)
    return closest


def predict_mistaken(word, vocab):
    # 0 если слово есть в словаре, 1 если нет
    return 0 if word in vocab else 1


def compute_word_proba(tokens):
    prob = 0
    seq = ['<start>'] + [t for t in tokens if t] + ['<end>']
    ngrams = get_ngrams(seq, 2)

    for ngram in ngrams:
        w1, w2 = ngram.split()
        if ngram in word_bigrams and w1 in word_unigrams:
            prob += np.log(word_bigrams[ngram] / word_unigrams[w1])
        else:
            prob += np.log(1e-10)  # Штраф за неизвестную биграмму

    return prob

### 6. Сбор известных тегов для слов из корпуса и подсчёт вероятности

Сделаем кэш, чтобы хранить теги для наших слов из словаря, во время подсчёта вероятности будем брать самый вероятный

In [14]:
word_tags_cache = defaultdict(Counter)

for filepath in conllu_files:
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith('#'):
                continue

            parts = line.split('\t')
            if len(parts) != 10:
                continue

            if parts[3] == 'PUNCT':
                continue

            word = parts[1].lower()
            tag = parse_conllu_features(parts[3], parts[5])

            word_tags_cache[word][tag] += 1

In [15]:
def compute_tag_proba(tags):
    prob = 0
    seq = ['<start>'] + tags + ['<end>']

    for i in range(len(seq) - 1):
        bigram = f"{seq[i]} {seq[i + 1]}"
        t1 = seq[i]

        if bigram in tag_bigrams and t1 in tag_unigrams:
            prob += np.log(tag_bigrams[bigram] / tag_unigrams[t1])
        else:
            prob += np.log(1e-10)  # Штраф за неизвестную биграмму

    return prob

### 7. Наш пайплайн исправления опечаток

In [16]:
def solve_spell_check(stanza_doc, lm_type='tag', limit=None):
    corrected_sentences = []

    # Берем срез предложений, если задан лимит
    sentences = stanza_doc.sentences[:limit] if limit else stanza_doc.sentences

    for sent in tqdm(sentences):
        token_options = []

        for token in sent.tokens:
            word_obj = token.words[0]
            text = word_obj.text
            lower = text.lower()

            # Пунктуация
            if word_obj.upos == 'PUNCT':
                token_options.append([(text, None)])
                continue

            # Проверка на ошибку
            if predict_mistaken(lower, vocab):
                preds = get_closest_hybrid_match(lower, X, vec)
                candidates = [p[0] for p in preds]
                candidates.append(lower)
                candidates = list(set(candidates))

                # Для каждого кандидата находим его возможные теги
                cand_options = []
                for cand in candidates:
                    possible_tags = word_tags_cache.get(cand, Counter())
                    if possible_tags:
                        best_tag = possible_tags.most_common(1)[0][0]
                        cand_options.append((cand, best_tag))
                    else:
                        original_tag = parse_conllu_features(
                            word_obj.upos, word_obj.feats)
                        cand_options.append((cand, original_tag))

                token_options.append(cand_options)

            # Слово правильное
            else:
                tag = parse_conllu_features(word_obj.upos, word_obj.feats)
                token_options.append([(lower, tag)])

        # Генерируем все комбинации предложений
        best_sent_str = ""
        best_score = -float('inf')

        for p in itertools.product(*token_options):
            # Формируем список слов для итогового предложения
            words = [x[0] for x in p]

            score = 0
            if lm_type == 'tag':
                tags_seq = [x[1] for x in p if x[1] is not None]
                score = compute_tag_proba(tags_seq)
            else:
                words_seq = [x[0] for x in p if x[1] is not None]
                score = compute_word_proba(words_seq)

            if score > best_score:
                best_score = score
                best_sent_str = " ".join(words)

        corrected_sentences.append(best_sent_str)

    return corrected_sentences

### 8. Метрики

In [17]:
def calculate_metrics_stanza(doc_true, doc_bad, predicted_strs):
    correct = 0
    total = 0
    total_mistaken = 0
    mistaken_fixed = 0
    total_correct = 0
    correct_broken = 0

    for i in range(len(doc_true.sentences)):
        true_tokens = [t.words[0].text.lower(
        ) for t in doc_true.sentences[i].tokens if t.words[0].upos != 'PUNCT']
        bad_tokens = [t.words[0].text.lower(
        ) for t in doc_bad.sentences[i].tokens if t.words[0].upos != 'PUNCT']

        # Предсказание мы сплитим просто по пробелам (так как мы собирали его join-ом)
        # Нужно тоже убрать пунктуацию, если она попала в строку
        pred_tokens = [t.strip('«»—…“”".,?!:')
                       for t in predicted_strs[i].split()]
        pred_tokens = [t for t in pred_tokens if t]  # убираем пустые

        min_len = min(len(true_tokens), len(bad_tokens), len(pred_tokens))

        for j in range(min_len):
            t = true_tokens[j]
            b = bad_tokens[j]
            p = pred_tokens[j]

            if t == p:
                correct += 1
            total += 1

            if t != b:  # Была ошибка
                total_mistaken += 1
                if t == p:
                    mistaken_fixed += 1
            else:  # Было верно
                total_correct += 1
                if t != p:
                    correct_broken += 1

    return {
        "total_accuracy": correct /
        total if total > 0 else 0,
        "fixed_mistakes": mistaken_fixed /
        total_mistaken if total_mistaken > 0 else 0,
        "broken_correct_words": correct_broken /
        total_correct if total_correct > 0 else 0}

### 9. Бейзлайн: без грамматических тегов

In [18]:
print("Running Correction with Word LM...")
preds_word = solve_spell_check(doc_bad, lm_type='word')

metrics_word = calculate_metrics_stanza(doc_true, doc_bad, preds_word)
print("\nMetrics (Word LM):")
print(metrics_word)

Running Correction with Word LM...


  0%|          | 0/915 [00:00<?, ?it/s]


Metrics (Word LM):
{'total_accuracy': 0.7350359138068635, 'fixed_mistakes': 0.37830858618463525, 'broken_correct_words': 0.19976401179941003}


### 10. Коррекция с учётом грамматики

In [19]:
print("Running Correction with Tag LM...")
preds_tag = solve_spell_check(doc_bad, lm_type='tag')

metrics_tag = calculate_metrics_stanza(doc_true, doc_bad, preds_tag)
print("\nMetrics (Tag LM):")
print(metrics_tag)

Running Correction with Tag LM...


  0%|          | 0/915 [00:00<?, ?it/s]


Metrics (Tag LM):
{'total_accuracy': 0.7362330407023144, 'fixed_mistakes': 0.2588766946417043, 'broken_correct_words': 0.17651917404129794}


### 11. Посмотрим на различия

In [20]:
print(f"{'Metric':<25} | {'Word LM':<10} | {'Tag LM':<10}")
print("-" * 50)
for k in metrics_word:
    print(f"{k:<25} | {metrics_word[k]:.4f}     | {metrics_tag[k]:.4f}")

print("\nExamples of differences:")
print("-" * 50)
count = 0
for i in range(len(preds_word)):
    if preds_word[i] != preds_tag[i]:
        orig_text = doc_bad.sentences[i].text
        true_text = doc_true.sentences[i].text

        print(f"Original:  {orig_text}")
        print(f"Word LM:   {preds_word[i]}")
        print(f"Tag LM:    {preds_tag[i]}")
        print(f"Reference: {true_text}")
        print("-" * 50)
        count += 1
        if count >= 20:
            break

Metric                    | Word LM    | Tag LM    
--------------------------------------------------
total_accuracy            | 0.7350     | 0.7362
fixed_mistakes            | 0.3783     | 0.2589
broken_correct_words      | 0.1998     | 0.1765

Examples of differences:
--------------------------------------------------
Original:  Симпатичнейшое шпионское устройство, такой себе гламурный фотоаппарат девушки Бонда - миниатюрная модель камеры Superheadz Clap Camera.
Word LM:   симпатичный чемпионское устройство , такой себе гламурные фотоаппарат девушки бонда - миниатюрная модель камеры superjob clap america .
Tag LM:    симпатичнейшое шпионское устройство , такой себе гламурный фотоаппарат девушки бонда - миниатюрная модель камеры superjob la america .
Reference: Симпатичнейшее шпионское устройство такой себе гламурный фотоаппарат девушки Бонда миниатюрная модель камеры Superheadz Clap Camera
--------------------------------------------------
Original:  Опофеозом дня для меня сегодня 

### 12. И что в итоге?

Стоит начать с того, что словарь, получившийся из Синтагруса, примерно в три раза меньше словаря Википедии, который был на семинаре. Поэтому качество в целом ниже независимо от метода.

Если сравнивать два метода, метод Word LM с семинара отработал лучше — он исправил больше ошибок. Однако он же и показал больше ложноположительных срабатываний, т.е. исправил правильные слова. Получается, что алгоритм с моделью на грамм. тегах более консервативный и склонен реже классифицировать слово как ошибку, тогда как словный алгоритм поступает наоборот.

Скорее всего, это объясняется природой данных, на которых мы оцениваемся. Дело в том, что словный алгоритм, как и было написано в задании, часто не учитывает конкретную форму слова и предлагает замену с правильным словом, но неправильной формой. Например, для опечатки *основая цель* словный алгоритм предсказывает правильное по семантике, но неверное по смыслу *основа*, а грамматический алгоритм наоборот: *сосновая* (форма правильная, слово — нет). Но при этом словный алгоритм часто ложно исправляет имена собственные. В целом, в данных намного больше кейсов, когда опечатка никак не влияет на определение формы, поэтому для данного датасета алгоритм, учитывающий грамматику, как будто нужен сильно меньше и, соответственно, справляется хуже.

## 2.  Symspell (5 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Он основан только на одной операции - удалении символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово  (обратите внимание, что для одного удаления может быть несколько правильных слов!) 
3) При исправлении слова с опечаткой сначала само слово проверятся по словарю удаления, а затем для этого слова генерируются все варианты удалений, и каждый вариант проверяется по словарю удалений. Если в словаре удалений таким образом находится совпадение, то соответствующее ему правильное слово становится исправлением.
Если совпадений несколько, то выбирается наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

### 1. Делаем словарь удалений

In [21]:
deletes_dict = defaultdict(set)


def get_deletes_list(word):
    if len(word) <= 1:
        return []
    return [word[:i] + word[i + 1:] for i in range(len(word))]

In [22]:
# У нас уже есть готовый словарь из 1 задания
for word in tqdm(vocab.keys()):
    # Генерируем варианты удалений для правильного слова
    deletes = get_deletes_list(word)
    for d in deletes:
        deletes_dict[d].add(word)

print(f"Size of deletes dict: {len(deletes_dict)}")

  0%|          | 0/121389 [00:00<?, ?it/s]

Size of deletes dict: 1017239


### 2. Собственно алгоритм

In [23]:
def symspell_correction(word):
    candidates = set()

    # Проверяем само слово в словаре удалений
    if word in deletes_dict:
        candidates.update(deletes_dict[word])

    # Генерируем удаления для ошибочного слова
    typo_deletes = get_deletes_list(word)
    for d in typo_deletes:
        if d in deletes_dict:
            candidates.update(deletes_dict[d])

    # Если кандидатов нет, возвращаем исходное слово
    if not candidates:
        return word

    # Выбираем наиболее вероятное слово из кандидатов по частоте в словаре
    return max(candidates, key=lambda w: vocab[w])

### 3. Применение алгоритма к нашему датасету

In [24]:
def solve_symspell_check(stanza_doc):
    corrected_sentences = []

    for sent in tqdm(stanza_doc.sentences):
        pred_tokens = []

        for token in sent.tokens:
            word_obj = token.words[0]
            text = word_obj.text
            lower = text.lower()

            # Пропускаем пунктуацию
            if word_obj.upos == 'PUNCT':
                pred_tokens.append(text)
                continue

            # Если слова нет в словаре - пытаемся исправить
            if predict_mistaken(lower, vocab):
                correction = symspell_correction(lower)
                pred_tokens.append(correction)
            else:
                pred_tokens.append(lower)

        # Собираем предложение обратно
        corrected_sentences.append(" ".join(pred_tokens))

    return corrected_sentences

### 4. Момент истины: запускаем и оцениваем

In [25]:
print("Running SymSpell correction...")
preds_symspell = solve_symspell_check(doc_bad)

metrics_symspell = calculate_metrics_stanza(doc_true, doc_bad, preds_symspell)

print("\nMetrics (SymSpell):")
for k, v in metrics_symspell.items():
    print(f"{k:<25} | {v:.4f}")

Running SymSpell correction...


  0%|          | 0/915 [00:00<?, ?it/s]


Metrics (SymSpell):
total_accuracy            | 0.7555
fixed_mistakes            | 0.3802
broken_correct_words      | 0.1759


Как видим, с учётом того, что у нас достаточно маленький словарь, очень неплохо! И точно лучше, чем любой алгоритм с LM, хоть со словами, хоть с тегами. Скорость — отдельная песня, все предложения обработались меньше, чем за секунду, это в разы быстрее алгоритмов с вероятностями и дорогими перестановками, которые работают больше десяти минут.

# Задание 3 (2 балла)

Используя любой из алгоритмов из семинара или домашки, детально проанализируйте получаемые ошибки. Улучшите алгоритм так, чтобы исправить ошибки. Улучшения в алгоритме должны быть общими, не привязанными к конкретным словам (например, словарь исключений не будет считаться). За каждое улучшение, которое исправляет 5+ ошибок вы получите 0.5 балла (максимум 2 в целом)

### 1. Считаем ошибки в абсолютных значениях, чтобы было от чего отталкиваться

In [26]:
def get_absolute_errors(doc_true, doc_bad, predicted_strs):
    total_mistaken = 0
    mistaken_fixed = 0
    total_correct = 0
    correct_broken = 0

    for i in range(len(doc_true.sentences)):
        true_tokens = [t.words[0].text.lower(
        ) for t in doc_true.sentences[i].tokens if t.words[0].upos != 'PUNCT']
        bad_tokens = [t.words[0].text.lower(
        ) for t in doc_bad.sentences[i].tokens if t.words[0].upos != 'PUNCT']

        pred_tokens = [t.strip('«»—…“”".,?!:')
                       for t in predicted_strs[i].split()]
        pred_tokens = [t for t in pred_tokens if t]

        min_len = min(len(true_tokens), len(bad_tokens), len(pred_tokens))

        for j in range(min_len):
            t = true_tokens[j]
            b = bad_tokens[j]
            p = pred_tokens[j]

            if t != b:  # Изначально была ошибка
                total_mistaken += 1
                if t == p:  # Исправили
                    mistaken_fixed += 1
            else:  # Изначально было верно
                total_correct += 1
                if t != p:  # Сломали
                    correct_broken += 1

    return total_mistaken, mistaken_fixed, total_correct, correct_broken

In [27]:
# Считаем для Word LM
tm, fixed_word, tc, broken_word = get_absolute_errors(
    doc_true, doc_bad, preds_word)
print(f"Word LM Results:")
print(f"Total Typos found: {tm}")
print(f"Typos Fixed:       {fixed_word}")
print(f"Correct Broken:    {broken_word}")

Word LM Results:
Total Typos found: 1549
Typos Fixed:       586
Correct Broken:    1693


In [28]:
# Считаем для Tag LM
tm, fixed_tag, tc, broken_tag = get_absolute_errors(
    doc_true, doc_bad, preds_tag)
print(f"Tag LM Results:")
print(f"Total Typos found: {tm}")
print(f"Typos Fixed:       {fixed_tag}")
print(f"Correct Broken:    {broken_tag}")

Tag LM Results:
Total Typos found: 1549
Typos Fixed:       401
Correct Broken:    1496


### 2. Гибридный алгоритм

Как мы видели при анализе результатов первого задания, словная модель часто правильно предсказывает слово, но ошибается с формой, тогда как модель на основе тегов умеет правильно выбирать форму, но либо промахивается со словом, либо, чаще всего, вовсе не видит ошибку. Попробуем реализовать гибридный алгоритм, в котором будет высчитываться объединённая вероятность Word LM и Tag LM (с некоторым весом каждого компонента). Тогда можно надеяться, что Word LM поможет предсказать правильное слово, а Tag LM — выбрать его правильную форму. Вес грамматической информации сделаем чуть выше.

In [29]:
def solve_hybrid_check(stanza_doc, gamma=1.0, limit=None):
    corrected_sentences = []

    loop = stanza_doc.sentences[:limit] if limit else stanza_doc.sentences

    for sent in tqdm(loop):
        token_options = []

        # Генерация кандидатов (как в задании 1)
        for token in sent.tokens:
            word_obj = token.words[0]
            text = word_obj.text
            lower = text.lower()

            if word_obj.upos == 'PUNCT':
                token_options.append([(text, None)])
                continue

            if predict_mistaken(lower, vocab):
                preds = get_closest_hybrid_match(lower, X, vec)
                candidates = [p[0] for p in preds]
                candidates.append(lower)
                candidates = list(set(candidates))

                # Присваиваем теги кандидатам
                cand_options = []
                for cand in candidates:
                    possible_tags = word_tags_cache.get(cand, Counter())
                    if possible_tags:
                        best_tag = possible_tags.most_common(1)[0][0]
                        cand_options.append((cand, best_tag))
                    else:
                        original_tag = parse_conllu_features(
                            word_obj.upos, word_obj.feats)
                        cand_options.append((cand, original_tag))

                token_options.append(cand_options)
            else:
                tag = parse_conllu_features(word_obj.upos, word_obj.feats)
                token_options.append([(lower, tag)])

        # Перебор комбинаций и гибридная оценка
        best_sent_str = ""
        best_score = -float('inf')

        for p in itertools.product(*token_options):
            words = [x[0] for x in p]

            # Подготовка последовательностей (без None/пунктуации)
            words_seq = [x[0] for x in p if x[1] is not None]
            tags_seq = [x[1] for x in p if x[1] is not None]

            log_p_word = compute_word_proba(words_seq)
            log_p_tag = compute_tag_proba(tags_seq)

            # Гибридная формула
            final_score = log_p_word + (gamma * log_p_tag)

            if final_score > best_score:
                best_score = final_score
                best_sent_str = " ".join(words)

        corrected_sentences.append(best_sent_str)

    return corrected_sentences

### 3. Запускаем и оцениваем

In [30]:
print("Running Hybrid Correction...")

preds_hybrid = solve_hybrid_check(doc_bad, gamma=2.0)

# Считаем метрики
metrics = calculate_metrics_stanza(doc_true, doc_bad, preds_hybrid)
print("\nMetrics (Hybrid):")
for k, v in metrics.items():
    print(f"{k:<25} | {v:.4f}")

# Абсолютные ошибки
tm, fixed_hybrid, tc, broken_hybrid = get_absolute_errors(
    doc_true, doc_bad, preds_hybrid)

print(f"\nHybrid Model Results:")
print(f"Total misspellings found: {tm}")
print(f"Misspelings Fixed:       {fixed_hybrid}")
print(f"Correct Broken:    {broken_hybrid}")

# Сравнение
diff_fixed = fixed_hybrid - fixed_word
# Положительное число = сломали меньше слов
diff_broken = broken_word - broken_hybrid

print("\nImprovement over Word LM:")
print(f"Additional Misspellings Fixed: {diff_fixed}")
print(f"Fewer Correct Broken:   {diff_broken}")
print(f"Total Net Improvement:  {diff_fixed + diff_broken}")

Running Hybrid Correction...


  0%|          | 0/915 [00:00<?, ?it/s]


Metrics (Hybrid):
total_accuracy            | 0.7488
fixed_mistakes            | 0.3912
broken_correct_words      | 0.1858

Hybrid Model Results:
Total misspellings found: 1549
Misspelings Fixed:       606
Correct Broken:    1575

Improvement over Word LM:
Additional Misspellings Fixed: 20
Fewer Correct Broken:   118
Total Net Improvement:  138


### 4. Смотрим, что улучшили

In [31]:
print("\nTop 20 Corrections (Hybrid vs Word LM):")
print("-" * 60)

count = 0
for i in range(len(preds_hybrid)):
    t_tokens = [t.words[0].text.lower()
                for t in doc_true.sentences[i].tokens if t.words[0].upos != 'PUNCT']
    p_word_tokens = [t.strip('«»—…“”".,?!:')
                     for t in preds_word[i].split() if t.strip('«»—…“”".,?!:')]
    p_hybrid_tokens = [t.strip('«»—…“”".,?!:')
                       for t in preds_hybrid[i].split() if t.strip('«»—…“”".,?!:')]

    # Ищем случаи, где Hybrid лучше Word LM

    min_len = min(len(t_tokens), len(p_word_tokens), len(p_hybrid_tokens))

    for j in range(min_len):
        true_w = t_tokens[j]
        word_lm_w = p_word_tokens[j]
        hybrid_w = p_hybrid_tokens[j]

        improved = False
        reason = ""

        # Word LM ошибся, Hybrid прав
        if word_lm_w != true_w and hybrid_w == true_w:
            improved = True
            reason = "Fixed typo/hallucination"

        if improved:
            print(f"Reason:    {reason}")
            print(f"Original:  {doc_bad.sentences[i].text}")
            print(f"Word LM:   ... {word_lm_w} ...")
            print(f"Hybrid:    ... {hybrid_w} ...")
            print(f"Correct:   ... {true_w} ...")
            print("-" * 60)
            count += 1
            break  # Одно исправление на предложение для наглядности

    if count >= 20:
        break


Top 20 Corrections (Hybrid vs Word LM):
------------------------------------------------------------
Reason:    Fixed typo/hallucination
Original:  Симпатичнейшое шпионское устройство, такой себе гламурный фотоаппарат девушки Бонда - миниатюрная модель камеры Superheadz Clap Camera.
Word LM:   ... чемпионское ...
Hybrid:    ... шпионское ...
Correct:   ... шпионское ...
------------------------------------------------------------
Reason:    Fixed typo/hallucination
Original:  Отвественность за реализацию, естественно, лежит на контрактных пивоварах.
Word LM:   ... контрактный ...
Hybrid:    ... контрактных ...
Correct:   ... контрактных ...
------------------------------------------------------------
Reason:    Fixed typo/hallucination
Original:  Начальнег зажог павзрослому: всю предудущую неделю ходил покрытый прыщами, а с понедельника слег - ветрянка.
Word LM:   ... павзрослому ...
Hybrid:    ... по-взрослому ...
Correct:   ... по-взрослому ...
--------------------------------------

Ура! Как видим, улучшения довольно существенные даже по сравнению со словным алгоритмом, который работал лучше грамматического. Нельзя сказать, что мы стали сильно лучше находить и исправлять ошибки (но +20 всё же есть), но мы точно стали меньше галлюцинировать, т.е. ломать правильное. Выходит, что ожидания оправдались, и мы смогли получить компромисс и взять лучшее от каждой модели: грамматическая модель позволяет меньше галлюцинировать, а словная — лучше находить реальные опечатки, причём вместе с грамматической даже лучше, чем самостоятельно.